In [32]:
from modeling import EncoderExtractor, __feature_shape__
from run_training_encoders import __model_file_pattern__, __feature_data__, __label__, split_train_test_valid, normalize_feature_data
import pandas as pd
import os

In [2]:
extractor = EncoderExtractor()

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
extractors = []
best_encoders = pd.read_csv("best_encoders.csv")
for fold in range(1,11):
    fold_best_encoders = best_encoders[best_encoders['fold'] == fold]
    filepaths = []
    for feature in __feature_shape__.keys():
        kfold_file, _, _, l2_rate, dropout, _, _ = fold_best_encoders[fold_best_encoders['feature'] == feature].iloc[0]
        filepath = os.path.join(kfold_file[:-4] + "_models", __model_file_pattern__.format(feature, l2_rate, 0.5, fold))
        filepaths.append(filepath)
    extractor = EncoderExtractor()
    extractor.load_encoders(filepaths)
    extractors.append(extractor)

Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-image-l2rate0.001-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-vein-l2rate1e-05-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-xyprojection-l2rate0.01-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-color-l2rate0.001-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-texture-l2rate0.001-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-fourier-l2rate0.1-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-shape-l2rate0.001-dropout0.5-fold1.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-image-l2rate0.001-dropout0.5-fold2.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-vein-l2rate1e-05-dropout0.5-fold2.h5
Loading encoder  Dataset_10FoldCV_indexed_models\ENCODER-xyprojection-l2rate0.01-dropout0.5-fold2.h5
Loading encoder  Dataset_10FoldCV_inde

In [16]:
import numpy as np
X = [np.load(file) for file in __feature_data__.values()]
y = np.load(__label__)

In [21]:
kfold = pd.read_csv(kfold_file)

In [30]:
encoders_representations = []

In [36]:
ys = []

In [37]:
for fold, extractor in zip(range(1,11), extractors):
    (X_train, y_train), (X_valid, y_valid), (X_test, y_test) = split_train_test_valid(list(__feature_data__.keys()), kfold, fold, X, y)
    
    #X_train_encoders_representation = extractor.extract(X_train)
    #X_valid_encoders_representation = extractor.extract(X_valid)
    #X_test_encoders_representation = extractor.extract(X_test)
    #encoders_representations.append([X_train_encoders_representation, X_valid_encoders_representation, X_test_encoders_representation])
    ys.append([y_train, y_valid, y_test])

In [46]:
from sklearn.svm import SVC
best_val = 0.0
for C in [1e-1, 1,10,100,1000,10000]:
    for gamma in [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2]:
        stop = False
        valid_accuracies, test_accuracies = [], []
        for fold in range(1,11):
            X_train, X_valid, X_test = encoders_representations[fold-1]
            y_train, y_valid, y_test = ys[fold-1]
            X_train, X_valid, X_test = normalize_feature_data("combine", X_train, X_valid, X_test)
            svm = SVC(C=C, gamma=gamma)
            svm.fit(X_train, y_train)
            val_acc = np.mean(y_valid == svm.predict(X_valid))
            test_acc = np.mean(y_test == svm.predict(X_test)) 
            if val_acc < 0.98: 
                stop=True
                break
            print("Fold {} - val_acc {} - test_acc {}".format(fold, val_acc, test_acc))
            valid_accuracies.append(val_acc)
            test_accuracies.append(test_acc)
        if stop: 
            continue
        print("C = {}, gamma = {}".format(C, gamma))
        print("Valid accuracy: {} +- {}".format(np.mean(valid_accuracies), np.std(valid_accuracies)))
        print("Test accuracy: {} +- {}".format(np.mean(test_accuracies), np.std(test_accuracies)))
        if np.mean(valid_accuracies) >= best_val:
            best_val = np.mean(valid_accuracies)
            best_C = C
            best_gamma = gamma

Fold 1 - val_acc 0.9947368421052631 - test_acc 0.9790575916230366
Fold 2 - val_acc 0.9895287958115183 - test_acc 0.9790575916230366
Fold 3 - val_acc 0.9842931937172775 - test_acc 0.9685863874345549
Fold 1 - val_acc 1.0 - test_acc 0.9947643979057592
Fold 2 - val_acc 1.0 - test_acc 0.9895287958115183
Fold 3 - val_acc 0.9947643979057592 - test_acc 0.9895287958115183
Fold 4 - val_acc 1.0 - test_acc 1.0
Fold 5 - val_acc 1.0 - test_acc 1.0
Fold 6 - val_acc 1.0 - test_acc 0.9895287958115183
Fold 7 - val_acc 0.9895287958115183 - test_acc 1.0
Fold 8 - val_acc 0.9947643979057592 - test_acc 0.9894736842105263
Fold 9 - val_acc 0.9947368421052631 - test_acc 0.9947368421052631
Fold 10 - val_acc 1.0 - test_acc 0.9894736842105263
C = 1, gamma = 0.001
Valid accuracy: 0.9973794433728299 +- 0.0035142116669062166
Test accuracy: 0.993703499586663 +- 0.004573818813996617
Fold 1 - val_acc 1.0 - test_acc 0.9947643979057592
Fold 2 - val_acc 1.0 - test_acc 0.9895287958115183
Fold 3 - val_acc 0.9895287958115183 

In [47]:
best_C, best_gamma

(10000, 5e-05)